In [4]:
import sys
sys.path.append('..')

In [5]:
from src.deberta_lm import DebertaLMHeadModel

In [6]:
model = DebertaLMHeadModel.from_pretrained('../runs/bruh2')

Some weights of the model checkpoint at ../runs/bruh2 were not used when initializing DebertaLMHeadModel: ['pooler.dense.weight', 'classifier.bias', 'pooler.dense.bias', 'classifier.weight']
- This IS expected if you are initializing DebertaLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaLMHeadModel were not initialized from the model checkpoint at ../runs/bruh2 and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
from transformers import MODEL_FOR_CAUSAL_LM_MAPPING, AutoTokenizer

In [16]:
MODEL_CONFIG_CLASSES = list(MODEL_FOR_CAUSAL_LM_MAPPING.keys())
tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

('bigbird_pegasus',
 'gpt_neo',
 'big_bird',
 'camembert',
 'xlm-roberta',
 'roberta',
 'bert',
 'openai-gpt',
 'gpt2',
 'transfo-xl',
 'xlnet',
 'xlm',
 'ctrl',
 'reformer',
 'bert-generation',
 'xlm-prophetnet',
 'prophetnet',
 'bart',
 'mbart',
 'pegasus',
 'marian',
 'blenderbot',
 'blenderbot-small',
 'megatron-bert')

In [2]:
from datasets import load_metric

In [20]:
bleu = load_metric('bleu')
bertscore = load_metric('bertscore')
rouge = load_metric('rouge')
meteor = load_metric('meteor')

Downloading: 4.63kB [00:00, 4.76MB/s]                   
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [21]:
from datasets import load_from_disk, load_metric, DatasetDict


In [38]:
loaded_data = load_from_disk('../storycloze/storycloze_valid')
train_valid = loaded_data.train_test_split(test_size=0.1, seed=690)
test = load_from_disk('../storycloze/storycloze_test')
datasets = DatasetDict({
    'train': train_valid['train'],
    'test': test,
    'validation': train_valid['test']}
)

In [10]:
# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-base')
tokenizer = AutoTokenizer.from_pretrained('gpt2')

In [13]:
tokenizer.special_tokens_map

{'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'}

In [20]:
tokenizer.add_special_tokens({'additional_special_tokens': ['[ENDING]']})

1

In [21]:
tokenizer

PreTrainedTokenizerFast(name_or_path='gpt2', vocab_size=50257, model_max_len=1024, is_fast=True, padding_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'additional_special_tokens': ['[ENDING]']})

In [12]:
tokenizer.convert_tokens_to_ids(tokenizer.eos_token)

50256

In [51]:
sep_token = tokenizer.special_tokens_map['sep_token']

def tokenize_function(examples):
    text = [' '.join((examples['InputSentence1'][i], examples['InputSentence2'][i],
                        examples['InputSentence3'][i], examples['InputSentence4'][i], sep_token,
                        examples['RandomFifthSentenceQuiz1'][i] if not examples['label'][i] else examples['RandomFifthSentenceQuiz2'][i]))
            for i in range(len(examples['InputSentence1']))]
    output = tokenizer(text, add_special_tokens=False)
    return output

In [52]:
tokenized_datasets = datasets.map(
    tokenize_function,
    batched=True
)

100%|██████████| 1/1 [00:00<00:00, 50.00ba/s]


In [43]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
)

  0%|          | 0/2 [00:00<?, ?ba/s]


TypeError: can only concatenate list (not "str") to list

In [59]:
print(bool(test[0]['label']))

True


In [55]:
tokenizer(sep_token, add_special_tokens=False)['input_ids'][0]

2

In [45]:
sep_token

'[SEP]'

In [99]:
tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
tokenizer.special_tokens_map

False

In [86]:
rouge = load_metric('rouge')

In [87]:
rouge.add(prediction='bruh tnhis isoneth ', reference='bruh aidk dk')

In [88]:
rougel = rouge.compute()['rougeL']

In [96]:
rougel.mid.fmeasure

0.3333333333333333

In [6]:
def run_bleu():
    metric = load_metric('sacrebleu')
    metric.add(prediction='the cat is bleu', reference=['the cat is blue'])
    return metric.compute()
def run_metric(metric_name):
    metric = load_metric(metric_name)
    metric.add(prediction='the cat is blue', reference='the cat is blue')
    return metric.compute()

In [8]:
run_metric('meteor')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ethan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


{'meteor': 0.9921875}